# Getting Started with [`Pyfhel`](https://dl.acm.org/doi/pdf/10.1145/3474366.3486923)

In this notebook, we will be exploring two FHE schemes, viz. `bfv` and `ckks` using Pyfhel along with supporting examples of mathematical operations on 1D array.

### Importing the libraries

In [1]:
import time

In [2]:
import numpy as np
from Pyfhel import PyCtxt, Pyfhel, PyPtxt

HE1 = Pyfhel() # creating an empty Pyfhel object

### [FHE Schemes](https://dl.acm.org/doi/pdf/10.1145/3474366.3486923)

Virtually all modern FHE schemes are based on (variants of) the Learning with Errors (LWE) hardness assumption and rely on a small amount of noise added during encryption to guarantee security. During homomorphic operations, this noise grows. This effect is negligible for additions, but very significant for multiplications. Should the noise grow too large, decryption would no longer produce correct results. Theoretically, a technique known as bootstrapping can be used to homomorphically reset the noise in a ciphertext. However, this can be computationally expensive and therefore is not frequently used in practice. Instead, schemes are instantiated with parameters large enough to allow the computation to complete without requiring bootstrapping. In this notebook, we explore a second generation FHE scheme, **Brakerski/Fan-Vercauteren (BFV)** and Fourth generation, **Cheon-Kim-Kim-Song (CKKS)** schemes implemented in SEAL library. 

### Exploring `Brakerski/Fan-Vercauteren (BFV)` scheme

Lets first define the parameters for `bfv` schemes,

In [3]:
bfv_params = {
    'scheme':'bfv',
    'n':2**13,
    't':65537,
    't_bits':20,
    'sec':128,
}

where,

- scheme : 'BFV' , can also be 'bfv'.
- 'n' :  2\**13, Polynomial modulus degree, the number of slots per plain text of elements to be encoded in single ciphertext in a 2 by n/2 rectangular matrix (Type 2^D for D in [10,16]
- 't' : 65537, Plain text modulus. Encrypted operations happen modulo t. Must be prime such that t-1 be divisible by 2^N.
- 't_bits' : Number of bits in t. Used to generate suitable value for t. Overrides t if specified. 
- 'sec' :128, Security parameter. The equivalentlength of AES key is bits. Sets the ciphertext modulus q, can be one of {128, 192, 256}. More means more security but slower computations.

**Generate context for `bfv` scheme**

In [4]:
HE1.contextGen(**bfv_params)

**Generate a pair of public/secret keys**

In [5]:
HE1.keyGen()

In [6]:
HE1.rotateKeyGen() #Allows rotation/shifting

In [7]:
HE1.relinKeyGen() #relinearization Key Generation

In [8]:
print("\n1. Pyfhel FHE context generation")
print(f"\t{HE1}")


1. Pyfhel FHE context generation
	<bfv Pyfhel obj at 0x7f5788395490, [pk:Y, sk:Y, rtk:Y, rlk:Y, contx(n=8192, t=1032193, sec=128, qi=[], scale=1.0, )]>


## Integer Array Encoding and Encryption

We will define two 1D integer array, encode and encrypt them.

In [9]:
arr1 = np.arange(bfv_params['n'], dtype=np.int64) # arr1 = [0,1,2..., n-1]
arr2 = np.array([-bfv_params['t']//2, -1, -1], dtype=np.int64) # arr2 = [-t//2, -1, -1] (length = 3)

In [10]:
print(arr1)
print(arr2)

[   0    1    2 ... 8189 8190 8191]
[-32769     -1     -1]


We now create a PyPtxt plaintext with the encoded arrays. 

In [11]:
ptxt1 = HE1.encodeInt(arr1)
ptxt2 = HE1.encodeInt(arr2)

In [12]:
print(ptxt1)
print(ptxt2)

<Pyfhel Plaintext at 0x7f57160b2040, scheme=bfv, poly=76277x^8191 + F8950x^8190..., is_ntt=->
<Pyfhel Plaintext at 0x7f57160b20c0, scheme=bfv, poly=C54C9x^8191 + A8DA8x^8190..., is_ntt=->


We now encrypt the plaintxt ptxt and returns the ciphertext.

In [13]:
ctxt1 = HE1.encryptPtxt(ptxt1)
ctxt2 = HE1.encryptPtxt(ptxt2)

In [14]:
print(ctxt1)
print(ctxt2)

<Pyfhel Ciphertext at 0x7f57160acb80, scheme=bfv, size=2/2, noiseBudget=146>
<Pyfhel Ciphertext at 0x7f57160b56d0, scheme=bfv, size=2/2, noiseBudget=146>


Once we finish the encoding and encryption steps. In the next steps we will show some operations.

### Mathematical Operations on Encrypted data

In [15]:
cSum = ctxt1 + ctxt2  # Additon

cSub = ctxt1 - ctxt2 # Substraction

cMul = ctxt1 * ctxt2 # Multiplication

cSq = ctxt1**2 # Square of the ciphertext

cNeg = -ctxt1 # Negate

cPow = ctxt1**3 # Power Operation

cRotR = ctxt1 >> 2 # rotation operation 2 steps, the encoded data is places in a n//2 by 2 matrix. These rotation applys independently to each of the rows.

cRotL = ctxt1 << 2 # rotation left 2 steps

cCom = ctxt1 == ctxt2 # Comparing the two cipher text

# Operation with plain text

cpSum = ctxt1 + ptxt2 

cpSub = ctxt1 - ptxt2

cpMul = ctxt1 * ptxt2

In [16]:
# All operations leads cipher text

print("Secure operations")
print(" Ciphertext-ciphertext: ")
print("->\tctxt1 + ctxt2 = cSum: ", cSum)
print("->\tctxt1 - ctxt2 = cSub: ", cSub)
print("->\tctxt1 * ctxt2 = cMul: ", cMul)
print(" Single ciphertext: ")
print("->\tctxt1**2      = cSq  : ", cSq)
print("->\t- ctxt1       = cNeg : ", cNeg)
print("->\tctxt1**3      = cPow : ", cPow)
print("->\tctxt1 >> 2    = cRotR: ", cRotR)
print("->\tctxt1 << 2    = cRotL: ", cRotL)
print("->\tctxt1 == ctxt2 = cCom: ", cCom)
print(" Ciphertext-plaintext: ")
print("->\tctxt1 + ptxt2 = cpSum: ", cpSum)
print("->\tctxt1 - ptxt2 = cpSub: ", cpSub)
print("->\tctxt1 * ptxt2 = cpMul: ", cpMul)

Secure operations
 Ciphertext-ciphertext: 
->	ctxt1 + ctxt2 = cSum:  <Pyfhel Ciphertext at 0x7f57160b5770, scheme=bfv, size=2/2, noiseBudget=145>
->	ctxt1 - ctxt2 = cSub:  <Pyfhel Ciphertext at 0x7f57160c2770, scheme=bfv, size=2/2, noiseBudget=145>
->	ctxt1 * ctxt2 = cMul:  <Pyfhel Ciphertext at 0x7f57160c2810, scheme=bfv, size=3/3, noiseBudget=114>
 Single ciphertext: 
->	ctxt1**2      = cSq  :  <Pyfhel Ciphertext at 0x7f57160c28b0, scheme=bfv, size=3/3, noiseBudget=114>
->	- ctxt1       = cNeg :  <Pyfhel Ciphertext at 0x7f57160c2900, scheme=bfv, size=2/2, noiseBudget=146>
->	ctxt1**3      = cPow :  <Pyfhel Ciphertext at 0x7f57160c2950, scheme=bfv, size=2/2, noiseBudget=81>
->	ctxt1 >> 2    = cRotR:  <Pyfhel Ciphertext at 0x7f57160c29a0, scheme=bfv, size=2/2, noiseBudget=142>
->	ctxt1 << 2    = cRotL:  <Pyfhel Ciphertext at 0x7f57160c29f0, scheme=bfv, size=2/2, noiseBudget=143>
->	ctxt1 == ctxt2 = cCom:  False
 Ciphertext-plaintext: 
->	ctxt1 + ptxt2 = cpSum:  <Pyfhel Ciphertext at 0x

In case of multiplication, ciphertext-ciphertext multiplication increases the size of the ploynomials representing the resulting ciphertext. To prevent this growth, the relinearization technique is used (typically right after the multiplication operation) to resude the size of the cipher text back to the minimal size. For this, a special type of public key called Relinearization key is used. 

In [17]:
print("\n Relinearization-> Right after each multiplication.")
print(f"cMul before relinearization (size {cMul.size()}): {cMul}")


 Relinearization-> Right after each multiplication.
cMul before relinearization (size 3): <Pyfhel Ciphertext at 0x7f57160c2810, scheme=bfv, size=3/3, noiseBudget=114>


In [18]:
HE1.relinearize(cMul)

In [19]:
print(f"cMul after relinearization (size {cMul.size()}): {cMul}")
print(f"cPow after 2 mult&relin rounds:  (size {cPow.size()}): {cPow}")

cMul after relinearization (size 2): <Pyfhel Ciphertext at 0x7f57160c2810, scheme=bfv, size=2/3, noiseBudget=114>
cPow after 2 mult&relin rounds:  (size 2): <Pyfhel Ciphertext at 0x7f57160c2950, scheme=bfv, size=2/2, noiseBudget=81>


We see that the size has decreased after relinearization.

Now, its time for decrypting and decoding results. 

Previously, we encoded and encrypted the integers by using `HE.encode()` and `HE.encrypt()` operations. The process can also be done in a single operation by using `HE.encryptInt()` operation. In order to decode and decrypt the result, we will be using `HE.decryptInt()`.

In [20]:
r1     = HE1.decryptInt(ctxt1)
r2     = HE1.decryptInt(ctxt2)
rccSum = HE1.decryptInt(cSum)
rccSub = HE1.decryptInt(cSub)
rccMul = HE1.decryptInt(cMul)
rcSq   = HE1.decryptInt(cSq  )
rcNeg  = HE1.decryptInt(cNeg )
rcPow  = HE1.decryptInt(cPow )
rcRotR = HE1.decryptInt(cRotR)
rcRotL = HE1.decryptInt(cRotL)
rcpSum = HE1.decryptInt(cpSum)
rcpSub = HE1.decryptInt(cpSub)
rcpMul = HE1.decryptInt(cpMul)

In [21]:
print(" Decrypting results")
print(" Original ciphertexts: ")
print("   ->\tctxt1 --(decr)--> ", r1)
print("   ->\tctxt2 --(decr)--> ", r2)
print(" Ciphertext-ciphertext Ops: ")
print("   ->\tctxt1 + ctxt2 = ccSum --(decr)--> ", rccSum)
print("   ->\tctxt1 - ctxt2 = ccSub --(decr)--> ", rccSub)
print("   ->\tctxt1 * ctxt2 = ccMul --(decr)--> ", rccMul)
print(" Single ciphertext: ")
print("   ->\tctxt1**2      = cSq   --(decr)--> ", rcSq  )
print("   ->\t- ctxt1       = cNeg  --(decr)--> ", rcNeg )
print("   ->\tctxt1**3      = cPow  --(decr)--> ", rcPow )
print("   ->\tctxt1 >> 2    = cRotR --(decr)--> ", rcRotR)
print("   ->\tctxt1 << 2    = cRotL --(decr)--> ", rcRotL)
print(" Ciphertext-plaintext ops: ")
print("   ->\tctxt1 + ptxt2 = cpSum --(decr)--> ", rcpSum)
print("   ->\tctxt1 - ptxt2 = cpSub --(decr)--> ", rcpSub)
print("   ->\tctxt1 * ptxt2 = cpMul --(decr)--> ", rcpMul)

 Decrypting results
 Original ciphertexts: 
   ->	ctxt1 --(decr)-->  [   0    1    2 ... 8189 8190 8191]
   ->	ctxt2 --(decr)-->  [-32769     -1     -1 ...      0      0      0]
 Ciphertext-ciphertext Ops: 
   ->	ctxt1 + ctxt2 = ccSum --(decr)-->  [-32769      0      1 ...   8189   8190   8191]
   ->	ctxt1 - ctxt2 = ccSub --(decr)-->  [32769     2     3 ...  8189  8190  8191]
   ->	ctxt1 * ctxt2 = ccMul --(decr)-->  [ 0 -1 -2 ...  0  0  0]
 Single ciphertext: 
   ->	ctxt1**2      = cSq   --(decr)-->  [     0      1      4 ... -32824 -16445    -64]
   ->	- ctxt1       = cNeg  --(decr)-->  [    0    -1    -2 ... -8189 -8190 -8191]
   ->	ctxt1**3      = cPow  --(decr)-->  [      0       1       8 ... -425556 -499460  507969]
   ->	ctxt1 >> 2    = cRotR --(decr)-->  [4094 4095    0 ... 8187 8188 8189]
   ->	ctxt1 << 2    = cRotL --(decr)-->  [   2    3    4 ... 8191 4096 4097]
 Ciphertext-plaintext ops: 
   ->	ctxt1 + ptxt2 = cpSum --(decr)-->  [-32769      0      1 ...   8189   8190   819

In this case, we encountered different mathematical operations using `bfv` scheme. 

Note: In case of comparing the two cipher text values, we do not need to decrypt the result since the result is Boolean for ciphertext comparison. 

## Exploring `Cheon-Kim-Kim-Song (CKKS)` scheme

Lets first define the parameters for `ckks` schemes,

In [22]:
HE2 = Pyfhel() # creating an empty Pyfhel object for different scheme

In [23]:
ckks_params = {
    'scheme': 'CKKS',
    'n': 2**14,         
    'scale': 2**30,
    'qi': [60, 30, 30, 30, 60]
}

where,

- scheme : ckks
- 'n' : Polynomial modulus degree. For ckks, n/2 values can be encoded in a single cipher text.
- 'scale' : All the encodings will use it for float -> fixed point conversion: x_fix = round(x_float * scale). You can use this as a default scale or use a different scale on each operation.
- 'qi' : number of bits of each prime in a chain. Intermediatye values should be close to log2(scale) for each operation, to have small rounding errors. 

**Generate context**

In [24]:
HE2.contextGen(**ckks_params)
HE2.keyGen() # generates a pair of public/secret keys
HE2.rotateKeyGen()

We will define two float arrays, encode and encrypt them,

In [25]:
arr_x = np.array([0.1,0.2,0.3], dtype=np.float64)
arr_y = np.array([-1.5, 2.3, 4.7], dtype = np.float64)

In [26]:
print(arr_x)
print(arr_y)

[0.1 0.2 0.3]
[-1.5  2.3  4.7]


In [27]:
# Encoding

ptxt_x = HE2.encodeFrac(arr_x) # creates a PyPtxt plain text 
ptxt_y = HE2.encodeFrac(arr_y)

In [28]:
print(ptxt_x)
print(ptxt_y)

<Pyfhel Plaintext at 0x7f57160cbec0, scheme=ckks, poly=?, is_ntt=Y, mod_level=0>
<Pyfhel Plaintext at 0x7f57160cb2c0, scheme=ckks, poly=?, is_ntt=Y, mod_level=0>


In [29]:
# Encrypting

ctxt_x = HE2.encryptPtxt(ptxt_x) # Encrypts the plaintext ptxt_x and returns a PyCtxt
ctxt_y = HE2.encryptPtxt(ptxt_y) #  Alternatively you can use HE.encryptFrac(arr_y)

In [30]:
print(ctxt_x)
print(ctxt_y)

<Pyfhel Ciphertext at 0x7f57160c5720, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
<Pyfhel Ciphertext at 0x7f57160c24a0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>


### Mathematical Operations

In [31]:
%timeit

cSum = ctxt_x + ctxt_y       # Addition

cSub = ctxt_x - ctxt_y       # Substraction

cMul = ctxt_x * ctxt_y       # Multiplication

cSq   = ctxt_x**2            # Square

cNeg  = -ctxt_x              # Negate
                            
#cPow  = ctxt_x**3          # pow Not supported in CKKS, you will get an error when you execute this one.

cRotR = ctxt_x >> 2          # Rotation

cRotL = ctxt_x << 2          # Rotation Left

cCom = ctxt_x == ctxt_x      # Comparison

# Ciphetext-plaintext ops
cpSum = ctxt_x + ptxt_y       # Addition with Plain text

cpSub = ctxt_x - ptxt_y       # Substraction with Plain text

cpMul = ctxt_x * ptxt_y       # Product with Plain text

In [32]:
print("Secure operations")
print(" Ciphertext-ciphertext: ")
print("->\tctxt_x + ctxt_y = cSum: ", cSum)
print("->\tctxt_x - ctxt_y = cSub: ", cSub)
print("->\tctxt_x * ctxt_y = cMul: ", cMul)
print(" Single ciphertext: ")
print("->\tctxt_x**2      = cSq  : ", cSq  )
print("->\t- ctxt_x       = cNeg : ", cNeg )
print("->\tctxt_x >> 4    = cRotR: ", cRotR)
print("->\tctxt_x << 4    = cRotL: ", cRotL)
print("->\tctxt_x == ctxt_y = cCom: ", cCom)

print(" Ciphertext-plaintext: ")
print("->\tctxt_x + ptxt_y = cpSum: ", cpSum)
print("->\tctxt_x - ptxt_y = cpSub: ", cpSub)
print("->\tctxt_x * ptxt_y = cpMul: ", cpMul)

Secure operations
 Ciphertext-ciphertext: 
->	ctxt_x + ctxt_y = cSum:  <Pyfhel Ciphertext at 0x7f57160caf90, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x - ctxt_y = cSub:  <Pyfhel Ciphertext at 0x7f57160b5770, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x * ctxt_y = cMul:  <Pyfhel Ciphertext at 0x7f57160c2770, scheme=ckks, size=3/3, scale_bits=60, mod_level=1>
 Single ciphertext: 
->	ctxt_x**2      = cSq  :  <Pyfhel Ciphertext at 0x7f57160c2810, scheme=ckks, size=3/3, scale_bits=60, mod_level=1>
->	- ctxt_x       = cNeg :  <Pyfhel Ciphertext at 0x7f57160c28b0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x >> 4    = cRotR:  <Pyfhel Ciphertext at 0x7f57160c2900, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x << 4    = cRotL:  <Pyfhel Ciphertext at 0x7f57160c29a0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
->	ctxt_x == ctxt_y = cCom:  True
 Ciphertext-plaintext: 
->	ctxt_x + ptxt_y = cpSum:  <Pyfhel Ciphertext at 0x7f5716

Like `bfv` scheme, we also do relinearization here in order to reduce the size of the product.

In [33]:
print("\n Relinearization-> Right after each multiplication.\n")

print(f"cMul before relinearization (size {cMul.size()}): {cMul}")


 Relinearization-> Right after each multiplication.

cMul before relinearization (size 3): <Pyfhel Ciphertext at 0x7f57160c2770, scheme=ckks, size=3/3, scale_bits=60, mod_level=1>


In [34]:
HE2.relinKeyGen()
HE2.relinearize(cMul)

In [35]:
print(f"ccMul after relinearization (size {cMul.size()}): {cMul}")

ccMul after relinearization (size 2): <Pyfhel Ciphertext at 0x7f57160c2770, scheme=ckks, size=2/3, scale_bits=60, mod_level=1>


### More complex mathematical operations

More complex operations with CKKS require keeping track of the CKKS scale. Operating with two CKKS ciphertexts (or a ciphertext or a plain text) requires them to have the same scale and same modulus level.

- Scale: Multiplications yield a new scale, which is the product of the scales of the two operands. To scale down a ciphertext, the HE.rescale_to_next(ctxt) function is used, which switches the modulus to the next one in the qi chain and divides the ciphertext by the previous modulus. Since this is the only scale-down operation, it is advised to use `scale_bits` with the same size as the intermediate moduli in HE.qi.

- Mod Switching: Switches to the next modulus in the qi chain, but without rescaling. This is achieved by the HE.mod_switch_to_next(ctxt) function. To ease the life of the user, Pyfhel provides `HE.align_mod_n_scale(this, other)`, which automatically does the rescaling and mod switching.

In the next case, we will compute mean squared error, treating the average of two ciphertexts as the true distribution.

In [36]:
#  1. Mean
c_mean = (ctxt_x + ctxt_y) / 2

#  2. MSE
c_mse_1 = ~((ctxt_x - c_mean)**2)
c_mse_2 = (~(ctxt_y - c_mean)**2)
c_mse = (c_mse_1 + c_mse_2)/ 3

#  3. Cumulative sum
c_mse += (c_mse << 1)
c_mse += (c_mse << 2)  # element 0 contains the result
print("\n Rescaling & Mod Switching.")
print("->\tMean: ", c_mean)
print("->\tMSE_1: ", c_mse_1)
print("->\tMSE_2: ", c_mse_2)
print("->\tMSE: ", c_mse)


 Rescaling & Mod Switching.
->	Mean:  <Pyfhel Ciphertext at 0x7f57160c3040, scheme=ckks, size=2/2, scale_bits=60, mod_level=1>
->	MSE_1:  <Pyfhel Ciphertext at 0x7f57160c30e0, scheme=ckks, size=2/3, scale_bits=60, mod_level=2>
->	MSE_2:  <Pyfhel Ciphertext at 0x7f57160c31d0, scheme=ckks, size=2/3, scale_bits=60, mod_level=2>
->	MSE:  <Pyfhel Ciphertext at 0x7f57160c3130, scheme=ckks, size=2/3, scale_bits=60, mod_level=3>


### Decrypting and decoding results

In [37]:
r_x    = HE2.decryptFrac(ctxt_x)
r_y    = HE2.decryptFrac(ctxt_y)
rcSum = HE2.decryptFrac(cSum)
rcSub = HE2.decryptFrac(cSub)
rcMul = HE2.decryptFrac(cMul)
rcSq   = HE2.decryptFrac(cSq  )
rcNeg  = HE2.decryptFrac(cNeg )
rcRotR = HE2.decryptFrac(cRotR)
rcRotL = HE2.decryptFrac(cRotL)
rcpSum = HE2.decryptFrac(cpSum)
rcpSub = HE2.decryptFrac(cpSub)
rcpMul = HE2.decryptFrac(cpMul)
rmean  = HE2.decryptFrac(c_mean)
rmse   = HE2.decryptFrac(c_mse)

In [38]:
_r = lambda x: np.round(x, decimals=3)
print(" Decrypting results")
print(" Original ciphertexts: ")
print("   ->\tctxt_x --(decr)--> ", _r(r_x))
print("   ->\tctxt_y --(decr)--> ", _r(r_y))
print(" Ciphertext-ciphertext Ops: ")
print("   ->\tctxt_x + ctxt_y = ccSum --(decr)--> ", _r(rccSum))
print("   ->\tctxt_x - ctxt_y = ccSub --(decr)--> ", _r(rccSub))
print("   ->\tctxt_x * ctxt_y = ccMul --(decr)--> ", _r(rccMul))
print(" Single ciphertext: ")
print("   ->\tctxt_x**2      = cSq   --(decr)--> ", _r(rcSq  ))
print("   ->\t- ctxt_x       = cNeg  --(decr)--> ", _r(rcNeg ))
print("   ->\tctxt_x >> 4    = cRotR --(decr)--> ", _r(rcRotR))
print("   ->\tctxt_x << 4    = cRotL --(decr)--> ", _r(rcRotL))
print(" Ciphertext-plaintext ops: ")
print("   ->\tctxt_x + ptxt_y = cpSum --(decr)--> ", _r(rcpSum))
print("   ->\tctxt_x - ptxt_y = cpSub --(decr)--> ", _r(rcpSub))
print("   ->\tctxt_x * ptxt_y = cpMul --(decr)--> ", _r(rcpMul))
print(" Mean Squared error: ")
print("   ->\tmean(ctxt_x, ctxt_y) = c_mean --(decr)--> ", _r(rmean))
print("   ->\tmse(ctxt_x, ctxt_y)  = c_mse  --(decr)--> ", _r(rmse))

 Decrypting results
 Original ciphertexts: 
   ->	ctxt_x --(decr)-->  [ 0.1  0.2  0.3 ... -0.   0.   0. ]
   ->	ctxt_y --(decr)-->  [-1.5  2.3  4.7 ...  0.  -0.  -0. ]
 Ciphertext-ciphertext Ops: 
   ->	ctxt_x + ctxt_y = ccSum --(decr)-->  [-32769      0      1 ...   8189   8190   8191]
   ->	ctxt_x - ctxt_y = ccSub --(decr)-->  [32769     2     3 ...  8189  8190  8191]
   ->	ctxt_x * ctxt_y = ccMul --(decr)-->  [ 0 -1 -2 ...  0  0  0]
 Single ciphertext: 
   ->	ctxt_x**2      = cSq   --(decr)-->  [ 0.01  0.04  0.09 ...  0.   -0.   -0.  ]
   ->	- ctxt_x       = cNeg  --(decr)-->  [-0.1 -0.2 -0.3 ...  0.  -0.  -0. ]
   ->	ctxt_x >> 4    = cRotR --(decr)-->  [-0.   0.   0.1 ...  0.  -0.  -0. ]
   ->	ctxt_x << 4    = cRotL --(decr)-->  [ 0.297 -0.001  0.    ...  0.     0.1    0.2  ]
 Ciphertext-plaintext ops: 
   ->	ctxt_x + ptxt_y = cpSum --(decr)-->  [-1.4  2.5  5.  ... -0.   0.   0. ]
   ->	ctxt_x - ptxt_y = cpSub --(decr)-->  [ 1.6 -2.1 -4.4 ... -0.   0.   0. ]
   ->	ctxt_x * ptxt_y =

# Comparison between `bfv` and `ckks` scheme 

CKKS uses approximate arithmetic instead of exact arithmetic, in the sense that once we finish computation we might get a slightly different result than if we did the computation directly. This means that if you encrypt 2 and 3, add their ciphertexts, and decrypt you might get something like 4.99 or 5.01 but not 5. Other schemes such as BFV are exact, which mean they will yield exactly 5. CKKS is more suited for arithmetic on real numbers, where we can have approximate but close results, while BFV is more suited for arithmetic on integers.


Now, when we transform a number into ciphertext, each text can be seen as a ploynomial with noise. When you perform encrypted multiplication, the size of the output polynomial grows based on the size of the two multiplicands, which increases its memory footprints and slows down operations. In order to keep cipher text sizes low, it is recommended to relinearize after each multiplication, effectively shrinking the ciphertext back to minimum size. Besides this, the noise budget inside the cipher text grows greatly when multiplying. Each cipher text is given a noise budget when first encrypted, which decreases irreversibly for each multiplication. If the noise budget reaches zero, the result in ciphertext cannot be decrypted correctly anymore. 

**Example for `bfv` scheme**

we have our pyfhel object for `bfv` scheme and the two integer arrays and their corresponding ciphertexts.

In [39]:
arr1 = np.array([1,-1,1], dtype = np.int64)
arr2 = np.array([1,1,-1], dtype = np.int64)
ctxt1 = HE1.encryptInt(arr1)
ctxt2 = HE1.encryptInt(arr2)

In [40]:
print("Pyfhel bfv object : ",HE1)
print("Integer 1 : ", arr1[:4])
print("Integer 2 : ", arr2[:4])
print("Ciphertext 1 : ", ctxt1)
print("Ciphertext 2 : ", ctxt2)

Pyfhel bfv object :  <bfv Pyfhel obj at 0x7f5788395490, [pk:Y, sk:Y, rtk:Y, rlk:Y, contx(n=8192, t=1032193, sec=128, qi=[], scale=1.0, )]>
Integer 1 :  [ 1 -1  1]
Integer 2 :  [ 1  1 -1]
Ciphertext 1 :  <Pyfhel Ciphertext at 0x7f57160a9f40, scheme=bfv, size=2/2, noiseBudget=146>
Ciphertext 2 :  <Pyfhel Ciphertext at 0x7f57160acb80, scheme=bfv, size=2/2, noiseBudget=146>


In [41]:
print("Secure Multiplication")
step = 0 

Secure Multiplication


To get the noise level, we can measure by using HE1.noise_level(ctxt). We need the private key in order to know the level.

In [42]:
lvl = HE1.noise_level(ctxt1)

In [43]:
while lvl > 0:
    print(f"\tStep {step}: noise_lvl {lvl}, res{HE1.decryptInt(ctxt1)[:3]}")
    step += 1
    ctxt1 *= ctxt2   #Multiply
    ctxt1 = ~(ctxt1) #Relinearising after each multiplication
    lvl = HE1.noise_level(ctxt1)
print(f"Final Step {step} : noise_lvl {lvl}, res{HE1.decryptInt(ctxt1)[:3]}")

	Step 0: noise_lvl 146, res[ 1 -1  1]
	Step 1: noise_lvl 114, res[ 1 -1 -1]
	Step 2: noise_lvl 82, res[ 1 -1  1]
	Step 3: noise_lvl 49, res[ 1 -1 -1]
	Step 4: noise_lvl 16, res[ 1 -1  1]
Final Step 5 : noise_lvl 0, res[323575 -11958 -69734]


**`ckks` scheme mulplicative depth**

In this case, we do not have a noise level operation to check the current noise, since the noise is considered as part of the encoding and adds up to a loss in precision of the encoded values. We can precisely control the maximum number of multiplications by setting qi (in parameters for ckks). 

In [44]:
n_mults = 10

HE2 = Pyfhel(key_gen=True, context_params={
    'scheme': 'CKKS',
    'n': 2**14,         # For CKKS, n/2 values can be encoded in a single ciphertext. 
    'scale': 2**30,     # Each multiplication grows the final scale
    'qi': [60]+ [30]*n_mults +[60] # Number of bits of each prime in the chain. 
                        # Intermediate prime sizes should be close to log2(scale).
                        # One per multiplication! More/higher qi means bigger 
                        #  ciphertexts and slower ops.
})
HE2.relinKeyGen()

In [45]:
arr_x = np.array([1, -1, 1], dtype=np.float64) 
arr_y = np.array([1, 1, -1], dtype=np.float64)
ctxt_x = HE2.encryptFrac(arr_x)
ctxt_y = HE2.encryptFrac(arr_y)

In [46]:
print("Pyfhel ckks object : ",HE2)
print("Number 1 : ", arr_x[:4])
print("Number 2 : ", arr_y[:4])
print("Ciphertext 1 : ", ctxt_x)
print("Ciphertext 2 : ", ctxt_y)

Pyfhel ckks object :  <ckks Pyfhel obj at 0x7f57160ca940, [pk:Y, sk:Y, rtk:-, rlk:Y, contx(n=16384, t=0, sec=128, qi=[60, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 60], scale=1073741824.0, )]>
Number 1 :  [ 1. -1.  1.]
Number 2 :  [ 1.  1. -1.]
Ciphertext 1 :  <Pyfhel Ciphertext at 0x7f57160c3ea0, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
Ciphertext 2 :  <Pyfhel Ciphertext at 0x7f57160c5720, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>


In [47]:
_r = lambda x : np.round(x, decimals=3)[:3]

In [48]:
print(f"Securely multiplying {n_mults} times!")
for step in range(1,n_mults+1):
    ctxt_x *= ctxt_y  #Multiply
    ctxt_x = ~ (ctxt_x) #relinearizing after multiplication!
    print(f"Step {step}: res{_r(HE2.decryptFrac(ctxt_x))}")
try:
    ctxt_x *= ctxt_y
except ValueError as e:
    assert str(e)=='scale out of bounds'
    print(f"If we multiply further we get: {str(e)}")

Securely multiplying 10 times!
Step 1: res[ 1. -1. -1.]
Step 2: res[ 1. -1.  1.]
Step 3: res[ 1. -1. -1.]
Step 4: res[ 1.001 -1.001  1.001]
Step 5: res[ 1.002 -1.001 -1.001]
Step 6: res[ 1.002 -1.002  1.002]
Step 7: res[ 1.003 -1.003 -1.003]
Step 8: res[ 1.006 -1.006  1.006]
Step 9: res[ 1.008 -1.008 -1.008]
Step 10: res[ 1.01 -1.01  1.01]
If we multiply further we get: scale out of bounds


# Conclusion

In this notebook, we explored two different schemes, viz. `bfv` and `ckks` and different mathematical operation they can perform using Pyfhel. We also compared the two schemes, one which is suitable for integer operation (`bfv`) and the other for real number operation (`ckks`) and investigated their multiplication depths. 

To conclude, we can say Pyfhel is a good tool to explore FHE. It can be used as a good teaching tool. By providing python-native abstraction layer on top of existing FHE implementations, Pyfhel makes working with FHE accessible to a signicantly wider audience.

In the next case, we will be exploring the machine learning use cases by taking the cipher text encrypted by using Pyfhel schemes. 